In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

yr2020_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2020.csv')
yr2020_df['fips'] = yr2020_df['fips'].fillna(0).astype(np.int64)
yr2020_df['date'] = pd.to_datetime(yr2020_df['date'], yearfirst=True)
yr2021_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2021.csv')
yr2021_df['fips'] = yr2021_df['fips'].fillna(0).astype(np.int64)
yr2021_df['date'] = pd.to_datetime(yr2021_df['date'], yearfirst=True)
yr2022_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2022.csv')
yr2022_df['fips'] = yr2022_df['fips'].fillna(0).astype(np.int64)
yr2022_df['date'] = pd.to_datetime(yr2022_df['date'], yearfirst=True)
us_covid_df = pd.concat([yr2020_df, yr2021_df, yr2022_df])
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
3043149,2022-12-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
3043150,2022-12-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
3043151,2022-12-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
3043152,2022-12-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
77327,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
77329,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
77331,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
77333,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
77335,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1834330,2022-12-06,Arkansas,Arkansas,5001,6743,93.00,5000,Arkansas,AR,Arkansas,State,3017804
1834332,2022-12-07,Arkansas,Arkansas,5001,6749,93.00,5000,Arkansas,AR,Arkansas,State,3017804
1834334,2022-12-08,Arkansas,Arkansas,5001,6753,93.00,5000,Arkansas,AR,Arkansas,State,3017804
1834336,2022-12-09,Arkansas,Arkansas,5001,6755,93.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
3043149,2022-12-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
3043150,2022-12-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
3043151,2022-12-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
3043152,2022-12-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2022-12-10') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
1054,2022-12-10,Snohomish,Washington,197666,1353.00,53061,WA,County,822083,24044.53,164.58
2106,2022-12-10,Cook,Illinois,1462932,15531.00,17031,IL,County,5150233,28405.16,301.56
3157,2022-12-10,Orange,California,747533,7594.00,6059,CA,County,3175692,23539.22,239.13
4207,2022-12-10,Maricopa,Arizona,1470301,18167.00,4013,AZ,County,4485414,32779.61,405.02
5257,2022-12-10,Los Angeles,California,3578389,34296.00,6037,CA,County,10039107,35644.50,341.62
...,...,...,...,...,...,...,...,...,...,...,...
3040121,2022-12-10,Wheeler,Oregon,281,3.00,41069,OR,County,1332,21096.10,225.23
3040910,2022-12-10,King,Texas,54,0.00,48269,TX,County,272,19852.94,0.00
3041668,2022-12-10,Esmeralda,Nevada,123,5.00,32009,NV,County,873,14089.35,572.74
3042422,2022-12-10,Loving,Texas,355,1.00,48301,TX,County,169,210059.17,591.72


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
1054,2022-12-10,Snohomish,Washington,197666,1353.00,53061,WA,County,822083,24044.53,164.58,164.58,24044.53
2106,2022-12-10,Cook,Illinois,1462932,15531.00,17031,IL,County,5150233,28405.16,301.56,301.56,28405.16
3157,2022-12-10,Orange,California,747533,7594.00,6059,CA,County,3175692,23539.22,239.13,239.13,23539.22
4207,2022-12-10,Maricopa,Arizona,1470301,18167.00,4013,AZ,County,4485414,32779.61,405.02,405.02,32779.61
5257,2022-12-10,Los Angeles,California,3578389,34296.00,6037,CA,County,10039107,35644.50,341.62,341.62,35644.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040121,2022-12-10,Wheeler,Oregon,281,3.00,41069,OR,County,1332,21096.10,225.23,225.23,21096.10
3040910,2022-12-10,King,Texas,54,0.00,48269,TX,County,272,19852.94,0.00,0.00,19852.94
3041668,2022-12-10,Esmeralda,Nevada,123,5.00,32009,NV,County,873,14089.35,572.74,572.74,14089.35
3042422,2022-12-10,Loving,Texas,355,1.00,48301,TX,County,169,210059.17,591.72,591.72,210059.17


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1617856,2022-12-10,Winston,Alabama,8931,138.00,1133,AL,County,23629,37796.78,584.03,584.03,37796.78,1
1919217,2022-12-10,Hale,Alabama,5455,109.00,1065,AL,County,14651,37232.95,743.98,743.98,37232.95,2
1321680,2022-12-10,Clay,Alabama,4894,90.00,1027,AL,County,13235,36977.71,680.02,680.02,36977.71,3
1194709,2022-12-10,Morgan,Alabama,44007,543.00,1103,AL,County,119679,36770.86,453.71,453.71,36770.86,4
1110306,2022-12-10,Franklin,Alabama,11495,148.00,1059,AL,County,31362,36652.64,471.91,471.91,36652.64,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537622,2022-12-10,Park,Wyoming,7580,151.00,56029,WY,County,29194,25964.24,517.23,517.23,25964.24,19
2595228,2022-12-10,Big Horn,Wyoming,2994,64.00,56003,WY,County,11790,25394.40,542.83,542.83,25394.40,20
2283374,2022-12-10,Lincoln,Wyoming,4826,32.00,56023,WY,County,19830,24336.86,161.37,161.37,24336.86,21
1831377,2022-12-10,Sublette,Wyoming,2282,28.00,56035,WY,County,9831,23212.29,284.81,284.81,23212.29,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
1497081,2022-12-10,Lowndes,Alabama,3109,82.00,1085,AL,County,9726,31965.86,843.10,843.10,31965.86,31,1
1495101,2022-12-10,Crenshaw,Alabama,4435,108.00,1041,AL,County,13772,32203.02,784.20,784.20,32203.02,26,2
631431,2022-12-10,Walker,Alabama,22684,476.00,1127,AL,County,63521,35711.02,749.36,749.36,35711.02,7,3
1919217,2022-12-10,Hale,Alabama,5455,109.00,1065,AL,County,14651,37232.95,743.98,743.98,37232.95,2,4
1613900,2022-12-10,Covington,Alabama,11183,266.00,1039,AL,County,37049,30184.35,717.97,717.97,30184.35,45,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612911,2022-12-10,Johnson,Wyoming,2284,21.00,56019,WY,County,8445,27045.59,248.67,248.67,27045.59,15,19
2225436,2022-12-10,Uinta,Wyoming,6272,43.00,56041,WY,County,20226,31009.59,212.60,212.60,31009.59,11,20
2283374,2022-12-10,Lincoln,Wyoming,4826,32.00,56023,WY,County,19830,24336.86,161.37,161.37,24336.86,21,21
1491141,2022-12-10,Albany,Wyoming,11384,54.00,56001,WY,County,38880,29279.84,138.89,138.89,29279.84,13,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,164.58,24044.53,32,15
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,164.58,24044.53,32,15
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,164.58,24044.53,32,15
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,164.58,24044.53,32,15
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,164.58,24044.53,32,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036170,2022-12-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
3036171,2022-12-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
3036172,2022-12-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
3036173,2022-12-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1611865,2020-03-27,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,584.03,37796.78,18,1,2.00,nan
1611866,2020-03-28,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,584.03,37796.78,18,1,0.00,nan
1611867,2020-03-29,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,584.03,37796.78,18,1,0.00,nan
1611868,2020-03-30,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,584.03,37796.78,18,1,0.00,nan
1611869,2020-03-31,Winston,Alabama,2,0.00,1133,AL,County,23629,8.46,0.00,584.03,37796.78,18,1,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2322498,2022-12-06,Crook,Wyoming,1538,31.00,56011,WY,County,7584,20279.54,408.76,421.94,20279.54,9,23,7.00,0.93
2322499,2022-12-07,Crook,Wyoming,1538,31.00,56011,WY,County,7584,20279.54,408.76,421.94,20279.54,9,23,0.00,0.93
2322500,2022-12-08,Crook,Wyoming,1538,32.00,56011,WY,County,7584,20279.54,421.94,421.94,20279.54,9,23,0.00,0.93
2322501,2022-12-09,Crook,Wyoming,1538,32.00,56011,WY,County,7584,20279.54,421.94,421.94,20279.54,9,23,0.00,0.93


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa['14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa['deaths'] = usa['deaths'].astype(int)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1491089,2020-03-26,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,843.10,31965.86,1,31,1.00,32.57,0.00,nan
1491090,2020-03-27,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,843.10,31965.86,1,31,0.00,32.57,0.00,nan
1491091,2020-03-28,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,843.10,31965.86,1,31,0.00,32.57,0.00,nan
1491092,2020-03-29,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,843.10,31965.86,1,31,0.00,24.50,0.00,nan
1491093,2020-03-30,Lowndes,Alabama,1,0,1085,AL,County,9726,10.28,0.00,843.10,31965.86,1,31,0.00,24.50,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624424,2022-12-06,Teton,Wyoming,11947,16,56039,WY,County,23464,50916.30,68.19,68.19,50916.30,23,1,36.00,4.21,0.00,0.00
624425,2022-12-07,Teton,Wyoming,11947,16,56039,WY,County,23464,50916.30,68.19,68.19,50916.30,23,1,0.00,4.21,0.00,0.00
624426,2022-12-08,Teton,Wyoming,11947,16,56039,WY,County,23464,50916.30,68.19,68.19,50916.30,23,1,0.00,4.21,0.00,0.00
624427,2022-12-09,Teton,Wyoming,11947,16,56039,WY,County,23464,50916.30,68.19,68.19,50916.30,23,1,0.00,4.21,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2022-12-10') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
755027,2022-12-10,Imperial,California,85019,1020,6025,CA,County,181215,46916.09,562.87,562.87,46916.09,1,1,0.00,111.86,0.00,0.14
114637,2022-12-10,Shasta,California,43176,690,6089,CA,County,180080,23976.01,383.16,383.16,23976.01,2,40,0.00,15.29,0.00,0.00
1832294,2022-12-10,Tuolumne,California,17404,208,6109,CA,County,54478,31946.84,381.81,381.81,31946.84,3,9,0.00,16.36,0.00,-0.07
373631,2022-12-10,San Bernardino,California,717925,8135,6071,CA,County,2180085,32931.06,373.15,373.15,32931.06,4,8,0.00,514.07,0.00,0.14
1727573,2022-12-10,Glenn,California,7578,99,6021,CA,County,28393,26689.68,348.68,348.68,26689.68,5,24,0.00,4.00,0.00,0.00
5257,2022-12-10,Los Angeles,California,3578389,34296,6037,CA,County,10039107,35644.50,341.62,341.62,35644.50,6,4,0.00,3398.57,0.00,10.00
2224349,2022-12-10,Tehama,California,17164,219,6103,CA,County,65084,26372.07,336.49,336.49,26372.07,7,25,0.00,11.71,0.00,0.07
1329560,2022-12-10,Inyo,California,5630,60,6027,CA,County,18039,31210.16,332.61,332.61,31210.16,8,11,0.00,5.21,0.00,0.00
206154,2022-12-10,Tulare,California,161352,1546,6107,CA,County,466195,34610.41,331.62,331.62,34610.41,9,5,0.00,109.79,0.00,0.07
2856256,2022-12-10,Lassen,California,11568,101,6035,CA,County,30573,37837.31,330.36,330.36,37837.31,10,3,0.00,3.50,0.00,0.14


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
755027,2022-12-10,Imperial,California,85019,1020,6025,CA,County,181215,46916.09,562.87,562.87,46916.09,1,1,0.00,111.86,0.00,0.14
1615820,2022-12-10,Kings,California,65122,464,6031,CA,County,152940,42580.10,303.39,303.39,42580.10,16,2,0.00,37.93,0.00,0.07
2856256,2022-12-10,Lassen,California,11568,101,6035,CA,County,30573,37837.31,330.36,330.36,37837.31,10,3,0.00,3.50,0.00,0.14
5257,2022-12-10,Los Angeles,California,3578389,34296,6037,CA,County,10039107,35644.50,341.62,341.62,35644.50,6,4,0.00,3398.57,0.00,10.00
206154,2022-12-10,Tulare,California,161352,1546,6107,CA,County,466195,34610.41,331.62,331.62,34610.41,9,5,0.00,109.79,0.00,0.07
75308,2022-12-10,Madera,California,53184,510,6039,CA,County,157327,33804.75,324.17,324.17,33804.75,12,6,0.00,28.64,0.00,0.07
470673,2022-12-10,Kern,California,297405,2579,6029,CA,County,900202,33037.58,286.49,286.49,33037.58,19,7,0.00,247.29,0.00,0.93
373631,2022-12-10,San Bernardino,California,717925,8135,6071,CA,County,2180085,32931.06,373.15,373.15,32931.06,4,8,0.00,514.07,0.00,0.14
1832294,2022-12-10,Tuolumne,California,17404,208,6109,CA,County,54478,31946.84,381.81,381.81,31946.84,3,9,0.00,16.36,0.00,-0.07
74299,2022-12-10,Fresno,California,313203,2899,6019,CA,County,999101,31348.48,290.16,290.16,31348.48,18,10,0.00,207.86,0.00,0.43


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
755027,2022-12-10,Imperial,California,85019,1020,6025,CA,County,181215,46916.09,562.87,562.87,46916.09,1,1,0.00,111.86,0.00,0.14
114637,2022-12-10,Shasta,California,43176,690,6089,CA,County,180080,23976.01,383.16,383.16,23976.01,2,40,0.00,15.29,0.00,0.00
1832294,2022-12-10,Tuolumne,California,17404,208,6109,CA,County,54478,31946.84,381.81,381.81,31946.84,3,9,0.00,16.36,0.00,-0.07
373631,2022-12-10,San Bernardino,California,717925,8135,6071,CA,County,2180085,32931.06,373.15,373.15,32931.06,4,8,0.00,514.07,0.00,0.14
1727573,2022-12-10,Glenn,California,7578,99,6021,CA,County,28393,26689.68,348.68,348.68,26689.68,5,24,0.00,4.00,0.00,0.00
5257,2022-12-10,Los Angeles,California,3578389,34296,6037,CA,County,10039107,35644.50,341.62,341.62,35644.50,6,4,0.00,3398.57,0.00,10.00
2224349,2022-12-10,Tehama,California,17164,219,6103,CA,County,65084,26372.07,336.49,336.49,26372.07,7,25,0.00,11.71,0.00,0.07
1329560,2022-12-10,Inyo,California,5630,60,6027,CA,County,18039,31210.16,332.61,332.61,31210.16,8,11,0.00,5.21,0.00,0.00
206154,2022-12-10,Tulare,California,161352,1546,6107,CA,County,466195,34610.41,331.62,331.62,34610.41,9,5,0.00,109.79,0.00,0.07
2856256,2022-12-10,Lassen,California,11568,101,6035,CA,County,30573,37837.31,330.36,330.36,37837.31,10,3,0.00,3.50,0.00,0.14


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'case_day', '14day_case_avg']]
top10_ca_case_df['case_day'] = top10_ca_case_df['case_day'].astype(int)
top10_ca_case_df['date'] =  pd.to_datetime(top10_ca_case_df['date'], infer_datetime_format=True)
top10_ca_case_df

,date,county,cases,current_crate,case_day,14day_case_avg
0,2020-03-20,Imperial,4,2.21,4,0.71
1,2020-03-21,Imperial,4,2.21,0,0.71
2,2020-03-22,Imperial,4,2.21,0,0.71
3,2020-03-23,Imperial,4,2.21,0,0.71
4,2020-03-24,Imperial,9,4.97,5,1.00
...,...,...,...,...,...,...
9971,2022-12-06,Kern,295863,32866.29,0,137.14
9972,2022-12-07,Kern,295863,32866.29,0,137.14
9973,2022-12-08,Kern,297405,33037.58,1542,247.29
9974,2022-12-09,Kern,297405,33037.58,0,247.29


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'death_day', '14day_death_avg']]
top10_ca_death_df['death_day'] = top10_ca_death_df['death_day'].astype(int)
top10_ca_death_df['date'] =  pd.to_datetime(top10_ca_death_df['date'], infer_datetime_format=True)
top10_ca_death_df

,date,county,deaths,current_drate,death_day,14day_death_avg
0,2020-03-20,Imperial,0,0.00,0,0.00
1,2020-03-21,Imperial,0,0.00,0,0.00
2,2020-03-22,Imperial,0,0.00,0,0.00
3,2020-03-23,Imperial,0,0.00,0,0.00
4,2020-03-24,Imperial,0,0.00,0,0.00
...,...,...,...,...,...,...
9933,2022-12-06,Lassen,99,323.82,0,0.00
9934,2022-12-07,Lassen,99,323.82,0,0.00
9935,2022-12-08,Lassen,101,330.36,2,0.14
9936,2022-12-09,Lassen,101,330.36,0,0.14


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)